## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200" align="middle"/>
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [26]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head(3)

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2


In [2]:
# target research
train_data['purchase'].value_counts()

0    83494
1     1040
Name: purchase, dtype: int64

In [28]:
# split data
X = train_data.drop(['purchase'], axis=1)
y = train_data['purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
train = X_train.join(y_train)
test  = X_test.join(y_test)

In [29]:
def score_local(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

def test_results_local(promotion_strategy):
    test_data = test
    df = test_data[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
    promos = promotion_strategy(df)
    score_df = test_data.iloc[np.where(promos == 'Yes')]    
    irr, nir = score(score_df)
    print("Nice job!  See how well your strategy worked on our test data below!")
    print()
    print('Your irr with this strategy is {:0.4f}.'.format(irr))
    print()
    print('Your nir with this strategy is {:0.2f}.'.format(nir))
    
    return irr, nir

def visualisation_mixed_target(df):
    purchase = df['purchase'].astype(int)
    xs       = df.drop(['purchase'], axis=1)
    
    print("Box Chart \n")
    for column in xs:
        x = xs[column]
        sns.boxplot(x=purchase, y=x);
    
    print("Joint Plot \n")
    for column in xs:
        x = xs[column]
        sns.jointplot(x=purchase, y=x, kind="hex");
    
    print("Pair Plot \n")
    sns.pairplot(df[['V1','V2','V3','V4','V5','V6','V7','purchase']],hue="purchase");  

def benchmark(df):
    df["Promotion"]="Yes"
    return df["Promotion"].values

def customer_targets(df):
    mask_purchase_treatment = np.logical_and(df["Promotion"]=="Yes",df["purchase"]==1)
    mask_purchase_control = np.logical_and(df["Promotion"]=="Yes",df["purchase"]==0)
    
    purchase_treatment = df[mask_purchase_treatment]
    purchase_control = df[mask_purchase_control]
    return purchase_treatment, purchase_control 
    

In [48]:
purchase_treatment, purchase_control = customer_targets(train_data)
purchase_control.describe().round(4)

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,41643.0000,41643.0,41643.0000,41643.0000,41643.0000,41643.0000,41643.0000,41643.0000,41643.0000
mean,62821.1704,0.0,1.5056,29.9722,0.0097,1.6757,2.3234,2.5045,1.7015
std,36303.3469,0.0,0.8686,5.0240,0.9982,0.4681,0.8392,1.1173,0.4576
min,8.0000,0.0,0.0000,7.1040,-1.6846,1.0000,1.0000,1.0000,1.0000
25%,31381.5000,0.0,1.0000,26.5782,-0.8188,1.0000,2.0000,2.0000,1.0000
50%,62664.0000,0.0,2.0000,29.9887,0.0470,2.0000,2.0000,3.0000,2.0000
75%,93936.5000,0.0,2.0000,33.3392,0.9128,2.0000,3.0000,4.0000,2.0000
max,126181.0000,0.0,3.0000,49.8237,1.6920,2.0000,4.0000,4.0000,2.0000


In [47]:
purchase_treatment.describe().round(4)

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,721.0000,721.0,721.0000,721.0000,721.0000,721.0000,721.0000,721.0000,721.0000
mean,64378.5284,1.0,1.4743,29.8602,-0.1216,1.8682,2.3994,2.4868,1.6838
std,36063.8509,0.0,0.8447,5.0494,0.9821,0.3385,0.8936,1.1194,0.4653
min,162.0000,1.0,0.0000,14.2616,-1.6846,1.0000,1.0000,1.0000,1.0000
25%,31694.0000,1.0,1.0000,26.3509,-0.9919,2.0000,2.0000,1.0000,1.0000
50%,64663.0000,1.0,2.0000,30.0048,-0.2127,2.0000,3.0000,3.0000,2.0000
75%,95439.0000,1.0,2.0000,33.0306,0.6531,2.0000,3.0000,3.0000,2.0000
max,126005.0000,1.0,3.0000,44.7193,1.6920,2.0000,4.0000,4.0000,2.0000


In [71]:
(((purchase_treatment.describe() - purchase_control.describe()).abs() / purchase_control.describe()).iloc[1:].fillna(0).drop(["purchase","ID"], axis=1)*100.0).round(2).astype(str)+"%"

,V1,V2,V3,V4,V5,V6,V7
mean,2.07%,0.37%,1356.0%,11.49%,3.27%,0.71%,1.04%
std,2.75%,0.5%,1.61%,27.7%,6.48%,0.19%,1.69%
min,0.0%,100.75%,-0.0%,0.0%,0.0%,0.0%,0.0%
25%,0.0%,0.86%,-21.15%,100.0%,0.0%,50.0%,0.0%
50%,0.0%,0.05%,552.56%,0.0%,50.0%,0.0%,0.0%
75%,0.0%,0.93%,28.46%,0.0%,0.0%,25.0%,0.0%
max,0.0%,10.25%,0.0%,0.0%,0.0%,0.0%,0.0%


In [ ]:
def no_promotion_and_purchase(df):
    
    return promotion
no_promotion_and_purchase(train_data)

In [30]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    promotion=benchmark(df)
    
    
    
    return promotion

In [18]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice
test_results_local(promotion_strategy)
test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0096.

Your nir with this strategy is -1132.20.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


<ipython-input-16-38f98a39b034>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Promotion"]="Yes"


(0.009593158278250108, -1132.1999999999998)